In [8]:
import json
import pandas as pd
from pathlib import Path
# Path to the results file
results_dir = Path("/cs/labs/guykatz/idopinto12/projects/loop_invariant_generation/RLInv/experiments")
exp_name = "oss_20b_low_hard"
model_name = "gpt-oss-20b"
baseline_name = "UAutomizer25 (0.3.0-dev-d790fec)"
k = 3
split_name = "hard"
results_path =results_dir / f"{exp_name}/{model_name}/{model_name}_results.json"
include_gen_time = False
# Load the JSON results
with open(results_path, "r") as f:
    gpt_oss_20b_results = json.load(f)

# Flatten results into a DataFrame
df = pd.json_normalize(gpt_oss_20b_results["results"])
n_samples = df.shape[0]
# Remove all columns that have 'path' in their column names (case-insensitive)
df_results = df.drop(columns=[col for col in df.columns if 'path' in col.lower() or "dir" in col.lower()])
df_results = df_results.drop(columns=["arch","report.invariant_usefulness_report.decision_reason","report.invariant_correctness_report.decision_reason"])
# Compute speedup_with_gen: baseline_time / report.total_time_taken
df_results["speedup_with_gen"] = df_results["baseline_time"] / pd.to_numeric(df_results["report.total_time_taken"], errors="coerce")
# Compute speedup_without_gen: baseline_time / report.verification_time_taken
df_results["speedup_without_gen"] = df_results["baseline_time"] / pd.to_numeric(df_results["report.verification_time_taken"], errors="coerce")

In [9]:
df_results.head()

,task_index,task_name,baseline_time,report.final_decision,report.decision_rule,report.target_assert.content,report.target_assert.line_number,report.candidate_invariant.content,report.candidate_invariant.line_number,report.syntactic_validation_result,report.invariant_correctness_report.decision,report.invariant_correctness_report.time_taken,report.invariant_usefulness_report.decision,report.invariant_usefulness_report.time_taken,report.total_time_taken,report.verification_time_taken,report.model_generation_time,speedup_with_gen,speedup_without_gen
0,0,freire2_unwindbound1_4,114.460649,UNKNOWN,DEC-PROP,(int)(4 * s) - 12 * r * r == 1,15,(int)(4 * s) - 12 * r * r == 1,8,True,TRUE,20.334210,TIMEOUT,114.460649,119.809544,114.460649,5.348895,0.955355,1.000000
1,1,geo3-ll_unwindbound1_2,43.729456,TRUE,DEC-PROP,z * x - x + a - az * y == 0,19,z * x - x + a - az * y == 0,19,True,TRUE,42.082924,TRUE,9.535251,43.196798,42.082924,1.113874,1.012331,1.039126
2,2,bresenham-ll_unwindbound10_2,478.011898,UNKNOWN,DEC-U,2 * yx - 2 * xy - X + (long long)2 * Y - v + 2...,24,2 * yx - 2 * xy - X + (long long)2 * Y - v + 2...,9,True,FALSE,4.402065,TRUE,53.838237,55.087087,53.838237,1.248850,8.677386,8.878669
3,3,cohencu-ll_valuebound5_9,47.204000,TRUE,DEC-PROP,2 * y * y - 3 * x * z - 18 * x - 10 * y + 3 * ...,16,2 * y * y - 3 * x * z - 18 * x - 10 * y + 3 * ...,8,True,TRUE,28.517899,TRUE,5.814938,30.167595,28.517899,1.649697,1.564725,1.655241
4,4,fermat1-ll_unwindbound10_4,153.856209,UNKNOWN,DEC-U,((long long)4 * A) == u * u - v * v - 2 * u + ...,27,((long long)4 * A) == u * u - v * v - 2 * u + ...,27,True,FALSE,94.268569,TRUE,4.895206,94.800740,94.268569,0.532171,1.622943,1.632105


In [ ]:
metrics = {
    "% Correct Invariant": None,
    "% Speedup": None,
    "Avg Speedup > 1": None,
    "Avg Speedup All": None,
}
speedup_col = "speedup_with_gen" if include_gen_time else "speedup_without_gen"
correct_invariants = df_results[df_results["report.invariant_correctness_report.decision"] == "TRUE"] # TODO: Think if to include correct but not useful
print(correct_invariants.shape)
correct_invariants.head()

(31, 19)


,task_index,task_name,baseline_time,report.final_decision,report.decision_rule,report.target_assert.content,report.target_assert.line_number,report.candidate_invariant.content,report.candidate_invariant.line_number,report.syntactic_validation_result,report.invariant_correctness_report.decision,report.invariant_correctness_report.time_taken,report.invariant_usefulness_report.decision,report.invariant_usefulness_report.time_taken,report.total_time_taken,report.verification_time_taken,report.model_generation_time,speedup_with_gen,speedup_without_gen
0,0,freire2_unwindbound1_4,114.460649,UNKNOWN,DEC-PROP,(int)(4 * s) - 12 * r * r == 1,15,(int)(4 * s) - 12 * r * r == 1,8,True,TRUE,20.334210,TIMEOUT,114.460649,119.809544,114.460649,5.348895,0.955355,1.000000
1,1,geo3-ll_unwindbound1_2,43.729456,TRUE,DEC-PROP,z * x - x + a - az * y == 0,19,z * x - x + a - az * y == 0,19,True,TRUE,42.082924,TRUE,9.535251,43.196798,42.082924,1.113874,1.012331,1.039126
3,3,cohencu-ll_valuebound5_9,47.204000,TRUE,DEC-PROP,2 * y * y - 3 * x * z - 18 * x - 10 * y + 3 * ...,16,2 * y * y - 3 * x * z - 18 * x - 10 * y + 3 * ...,8,True,TRUE,28.517899,TRUE,5.814938,30.167595,28.517899,1.649697,1.564725,1.655241
6,6,sll-queue-1_12,461.759118,TRUE,DEC-PROP,head->next->next != NULL,29,status >= 3 && head->next->next != NULL,29,True,TRUE,50.500510,TRUE,175.192380,176.985663,175.192380,1.793283,2.609020,2.635726
8,8,egcd2-ll_unwindbound50_6,51.606540,TRUE,DEC-PROP,p * x + r * y == a,38,p * x + r * y == a,38,True,TRUE,33.313507,TRUE,5.251292,34.247274,33.313507,0.933767,1.506880,1.549118


In [39]:
mask_speedup_gt1 = (
    (df_results[speedup_col] > 1)
    & (df_results["report.final_decision"] != "UNKNOWN")
    & (df_results["report.invariant_correctness_report.decision"] == "TRUE")
)
speedup_gt1 =df_results[mask_speedup_gt1]
print(speedup_gt1.shape)
speedup_gt1.head()

(26, 19)


,task_index,task_name,baseline_time,report.final_decision,report.decision_rule,report.target_assert.content,report.target_assert.line_number,report.candidate_invariant.content,report.candidate_invariant.line_number,report.syntactic_validation_result,report.invariant_correctness_report.decision,report.invariant_correctness_report.time_taken,report.invariant_usefulness_report.decision,report.invariant_usefulness_report.time_taken,report.total_time_taken,report.verification_time_taken,report.model_generation_time,speedup_with_gen,speedup_without_gen
1,1,geo3-ll_unwindbound1_2,43.729456,TRUE,DEC-PROP,z * x - x + a - az * y == 0,19,z * x - x + a - az * y == 0,19,True,TRUE,42.082924,TRUE,9.535251,43.196798,42.082924,1.113874,1.012331,1.039126
3,3,cohencu-ll_valuebound5_9,47.204000,TRUE,DEC-PROP,2 * y * y - 3 * x * z - 18 * x - 10 * y + 3 * ...,16,2 * y * y - 3 * x * z - 18 * x - 10 * y + 3 * ...,8,True,TRUE,28.517899,TRUE,5.814938,30.167595,28.517899,1.649697,1.564725,1.655241
6,6,sll-queue-1_12,461.759118,TRUE,DEC-PROP,head->next->next != NULL,29,status >= 3 && head->next->next != NULL,29,True,TRUE,50.500510,TRUE,175.192380,176.985663,175.192380,1.793283,2.609020,2.635726
8,8,egcd2-ll_unwindbound50_6,51.606540,TRUE,DEC-PROP,p * x + r * y == a,38,p * x + r * y == a,38,True,TRUE,33.313507,TRUE,5.251292,34.247274,33.313507,0.933767,1.506880,1.549118
9,9,dijkstra-u_valuebound2_2,156.967098,TRUE,DEC-PROP,p * p + r * q == n * q,13,p * p + r * q == n * q,13,True,TRUE,123.542019,TRUE,11.073682,124.073379,123.542019,0.531361,1.265115,1.270556


In [48]:
df_speedup_all = df_results.copy()
mask_no_speedup = ~mask_speedup_gt1
df_speedup_all.loc[mask_no_speedup, speedup_col] = 1
df_speedup_all.head()
# df_speedup_all.head()


,task_index,task_name,baseline_time,report.final_decision,report.decision_rule,report.target_assert.content,report.target_assert.line_number,report.candidate_invariant.content,report.candidate_invariant.line_number,report.syntactic_validation_result,report.invariant_correctness_report.decision,report.invariant_correctness_report.time_taken,report.invariant_usefulness_report.decision,report.invariant_usefulness_report.time_taken,report.total_time_taken,report.verification_time_taken,report.model_generation_time,speedup_with_gen,speedup_without_gen
0,0,freire2_unwindbound1_4,114.460649,UNKNOWN,DEC-PROP,(int)(4 * s) - 12 * r * r == 1,15,(int)(4 * s) - 12 * r * r == 1,8,True,TRUE,20.334210,TIMEOUT,114.460649,119.809544,114.460649,5.348895,0.955355,1.000000
1,1,geo3-ll_unwindbound1_2,43.729456,TRUE,DEC-PROP,z * x - x + a - az * y == 0,19,z * x - x + a - az * y == 0,19,True,TRUE,42.082924,TRUE,9.535251,43.196798,42.082924,1.113874,1.012331,1.039126
2,2,bresenham-ll_unwindbound10_2,478.011898,UNKNOWN,DEC-U,2 * yx - 2 * xy - X + (long long)2 * Y - v + 2...,24,2 * yx - 2 * xy - X + (long long)2 * Y - v + 2...,9,True,FALSE,4.402065,TRUE,53.838237,55.087087,53.838237,1.248850,8.677386,1.000000
3,3,cohencu-ll_valuebound5_9,47.204000,TRUE,DEC-PROP,2 * y * y - 3 * x * z - 18 * x - 10 * y + 3 * ...,16,2 * y * y - 3 * x * z - 18 * x - 10 * y + 3 * ...,8,True,TRUE,28.517899,TRUE,5.814938,30.167595,28.517899,1.649697,1.564725,1.655241
4,4,fermat1-ll_unwindbound10_4,153.856209,UNKNOWN,DEC-U,((long long)4 * A) == u * u - v * v - 2 * u + ...,27,((long long)4 * A) == u * u - v * v - 2 * u + ...,27,True,FALSE,94.268569,TRUE,4.895206,94.800740,94.268569,0.532171,1.622943,1.000000


,task_index,task_name,baseline_time,report.final_decision,report.decision_rule,report.target_assert.content,report.target_assert.line_number,report.candidate_invariant.content,report.candidate_invariant.line_number,report.syntactic_validation_result,report.invariant_correctness_report.decision,report.invariant_correctness_report.time_taken,report.invariant_usefulness_report.decision,report.invariant_usefulness_report.time_taken,report.total_time_taken,report.verification_time_taken,report.model_generation_time,speedup_with_gen,speedup_without_gen
0,0,freire2_unwindbound1_4,114.460649,UNKNOWN,DEC-PROP,(int)(4 * s) - 12 * r * r == 1,15,(int)(4 * s) - 12 * r * r == 1,8,True,TRUE,20.334210,TIMEOUT,114.460649,119.809544,114.460649,5.348895,0.955355,1.000000
1,1,geo3-ll_unwindbound1_2,43.729456,TRUE,DEC-PROP,z * x - x + a - az * y == 0,19,z * x - x + a - az * y == 0,19,True,TRUE,42.082924,TRUE,9.535251,43.196798,42.082924,1.113874,1.012331,1.039126
2,2,bresenham-ll_unwindbound10_2,478.011898,UNKNOWN,DEC-U,2 * yx - 2 * xy - X + (long long)2 * Y - v + 2...,24,2 * yx - 2 * xy - X + (long long)2 * Y - v + 2...,9,True,FALSE,4.402065,TRUE,53.838237,55.087087,53.838237,1.248850,8.677386,1.000000
3,3,cohencu-ll_valuebound5_9,47.204000,TRUE,DEC-PROP,2 * y * y - 3 * x * z - 18 * x - 10 * y + 3 * ...,16,2 * y * y - 3 * x * z - 18 * x - 10 * y + 3 * ...,8,True,TRUE,28.517899,TRUE,5.814938,30.167595,28.517899,1.649697,1.564725,1.655241
4,4,fermat1-ll_unwindbound10_4,153.856209,UNKNOWN,DEC-U,((long long)4 * A) == u * u - v * v - 2 * u + ...,27,((long long)4 * A) == u * u - v * v - 2 * u + ...,27,True,FALSE,94.268569,TRUE,4.895206,94.800740,94.268569,0.532171,1.622943,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,67,hard-u_5,600.000000,UNKNOWN,DEC-U,d == B * p,16,d == B * p,16,True,TIMEOUT,600.000000,TRUE,6.698870,601.727530,600.000000,1.727530,0.997129,1.000000
68,68,freire2_valuebound10_2,600.000000,UNKNOWN,DEC-U,((4 * r * r * r - 6 * r * r + 3 * r) + (int)(4...,7,((4 * r * r * r - 6 * r * r + 3 * r) + (int)(4...,7,True,TIMEOUT,600.000000,TRUE,63.594556,601.665073,600.000000,1.665073,0.997233,1.000000
69,69,eureka_01-1_1,600.000000,UNKNOWN,DEC-U,distance[i] >= 0,41,distance[i] >= 0,18,True,FALSE,8.054239,TIMEOUT,600.000000,601.151515,600.000000,1.151515,0.998084,1.000000
70,70,num_conversion_1_1,30.012025,UNKNOWN,DEC-U,x == y,14,y == (x & (((unsigned char)1 << c) - 1)),7,True,TIMEOUT,30.012025,TRUE,27.110751,31.830366,30.012025,1.818341,0.942874,1.000000


In [53]:
import numpy as np
percent_correct = correct_invariants.shape[0] / n_samples
print(f"Percent correct: {percent_correct * 100:.2f}%")
percent_speedup_gt1 = speedup_gt1.shape[0] / n_samples
print(f"Percent speedup > 1: {percent_speedup_gt1 * 100:.2f}%")
avg_speedup_gt1 = speedup_gt1[speedup_col].mean()
print(f"Avg speedup > 1: {avg_speedup_gt1:.2f}")
avg_speedup_all = df_speedup_all[speedup_col].mean()
print(f"Avg speedup all: {avg_speedup_all:.2f}")
metrics = {
    "% Correct Invariant": float(np.round(percent_correct * 100, 2)),
    "% Speedup": float(np.round(percent_speedup_gt1 * 100, 2)),
    "Avg Speedup > 1": float(np.round(avg_speedup_gt1, 2)),
    "Avg Speedup All": float(np.round(avg_speedup_all, 2)),
}
metrics

Percent correct: 43.06%
Percent speedup > 1: 36.11%
Avg speedup > 1: 5.80
Avg speedup all: 2.73


{'% Correct Invariant': 43.06,
 '% Speedup': 36.11,
 'Avg Speedup > 1': 5.8,
 'Avg Speedup All': 2.73}

In [54]:
def calculate_metrics(df, include_gen_time: bool = False):
    n_samples = df.shape[0]
    speedup_col = "speedup_with_gen" if include_gen_time else "speedup_without_gen"
    correct_invariants = df[df["report.invariant_correctness_report.decision"] == "TRUE"] # TODO: Think if to include correct but not useful
    mask_speedup_gt1 = (
    (df[speedup_col] > 1)
    & (df["report.final_decision"] != "UNKNOWN")
    & (df["report.invariant_correctness_report.decision"] == "TRUE")
    )
    speedup_gt1 =df[mask_speedup_gt1]
    df_speedup_all = df.copy()
    mask_no_speedup = ~mask_speedup_gt1
    df_speedup_all.loc[mask_no_speedup, speedup_col] = 1
    percent_correct = correct_invariants.shape[0] / n_samples
    percent_speedup_gt1 = speedup_gt1.shape[0] / n_samples
    avg_speedup_gt1 = speedup_gt1[speedup_col].mean()
    avg_speedup_all = df_speedup_all[speedup_col].mean()
    metrics = {
    "% Correct Invariant": float(np.round(percent_correct * 100, 2)),
    "% Speedup": float(np.round(percent_speedup_gt1 * 100, 2)),
    "Avg Speedup > 1": float(np.round(avg_speedup_gt1, 2)),
    "Avg Speedup All": float(np.round(avg_speedup_all, 2)),
    }
    return metrics

In [55]:
calculate_metrics(df_results)

{'% Correct Invariant': 43.06,
 '% Speedup': 36.11,
 'Avg Speedup > 1': 5.8,
 'Avg Speedup All': 2.73}

In [56]:
calculate_metrics(df_results, include_gen_time=True)

{'% Correct Invariant': 43.06,
 '% Speedup': 34.72,
 'Avg Speedup > 1': 5.19,
 'Avg Speedup All': 2.45}

###
### Invariant is **verified-correct** when:
### - $V(P, \{\}, q) = \mathrm{TRUE}$
###
### Invariant is **valid for speedup count** when:
###
### - **If `DEC-FALSE`**:
###     - $V(P, \{q\}, p^*) = \mathrm{FALSE}$ (before the baseline timeout)
###     - But $V(P, \{\}, q)$ can be TRUE or FALSE. If it is **FALSE** and speedup $> 1$, it is an *invalid* invariant for speedup count.
###     - Only if $V(P, \{\}, q) = \mathrm{TRUE}$ is it a *valid* invariant for speedup count.
### - **If `DEC-PROP`**:
###     - $V(P, \{\}, q) = \mathrm{TRUE}$
###     - But we can't take it into account for speedup count if the final decision is UNKNOWN.
### - **If `DEC-U`**:
###     - $V(P, \{\}, q)$ is FALSE or UNKNOWN.
###     - So it is *not* a valid invariant for speedup count.
###
### **Therefore, we must count only when:**
### - `final_decision = TRUE` **and** $V(P, \{\}, q) = \mathrm{TRUE}$


In [19]:
# Show only the rows where the invariant correctness is FALSE and speedup_col > 1
df_x = df_results[
    (df_results["report.invariant_correctness_report.decision"] == "FALSE")
    & (df_results[speedup_col] > 1)
]
# 9


In [20]:
df_x.head(1)

,task_index,task_name,baseline_time,report.final_decision,report.decision_rule,report.target_assert.content,report.target_assert.line_number,report.candidate_invariant.content,report.candidate_invariant.line_number,report.syntactic_validation_result,report.invariant_correctness_report.decision,report.invariant_correctness_report.time_taken,report.invariant_usefulness_report.decision,report.invariant_usefulness_report.time_taken,report.total_time_taken,report.verification_time_taken,report.model_generation_time,speedup_with_gen,speedup_without_gen
2,2,bresenham-ll_unwindbound10_2,478.011898,UNKNOWN,DEC-U,2 * yx - 2 * xy - X + (long long)2 * Y - v + 2...,24,2 * yx - 2 * xy - X + (long long)2 * Y - v + 2...,9,True,FALSE,4.402065,TRUE,53.838237,55.087087,53.838237,1.24885,8.677386,8.878669


In [ ]:
# V(P, {}, q) = FALSE and V(P,{q}, p*) = TRUE

In [23]:
# Show only the rows where the invariant correctness is FALSE and speedup_col > 1
df_x2 = df_results[
    (df_results["report.invariant_usefulness_report.decision"] == "FALSE")
    & (df_results[speedup_col] > 1)
]
# 9
df_x2.head()


,task_index,task_name,baseline_time,report.final_decision,report.decision_rule,report.target_assert.content,report.target_assert.line_number,report.candidate_invariant.content,report.candidate_invariant.line_number,report.syntactic_validation_result,report.invariant_correctness_report.decision,report.invariant_correctness_report.time_taken,report.invariant_usefulness_report.decision,report.invariant_usefulness_report.time_taken,report.total_time_taken,report.verification_time_taken,report.model_generation_time,speedup_with_gen,speedup_without_gen
37,37,egcd3-ll_unwindbound10_5,66.674998,FALSE,DEC-FALSE,p * x - q * x + r * y - s * y == a,45,a == x && b == y && p == 1 && q == 0 && r == 0...,12,True,FALSE,9.165034,FALSE,12.70847,15.584754,12.70847,2.876284,4.278219,5.246501
